In [ ]:
"""
===========================================================
Analysis of growth curves of CRISPRi library
===========================================================
"""

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy import stats
from scipy import integrate
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from scipy.stats import linregress
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from sklearn.linear_model import LinearRegression
import os

In [ ]:
"""
===========================================================
functions
===========================================================
"""

In [ ]:
# Define all functions

def remove_samples(df, samples_to_remove):
    return df[~df['Sample'].isin(samples_to_remove)]

def convert_time_to_minutes(time_str):
        try:
            if 'min' in time_str:
                hours, minutes = time_str.split(' h ')
                minutes = int(minutes.split(' min')[0])
            else:
                hours = time_str.split(' h')[0]
                minutes = 0
            hours = int(hours)
            total_minutes = hours * 1 + minutes/60
            return total_minutes
        except ValueError:
            return pd.NA  # Return NaN if the time string is not in the expected format
        
def apply_savgol_filter(group, window_size, polyorder=1):
    """
    Apply Savitzky-Golay filter to smooth the 'OD' values in each group.
    
    Parameters:
        group (DataFrame): Grouped data by 'Sample'.
        window_size (int): The length of the filter window. Must be a positive odd integer.
        polyorder (int): The order of the polynomial used to fit the samples. Must be less than window_size.
    
    Returns:
        DataFrame: The original group with an additional 'Smoothed_OD' column.
    """
    # Apply Savitzky-Golay filter to the 'OD' column
    group['Smoothed_OD'] = savgol_filter(group['OD'], window_size, polyorder, mode='nearest')
    return group


def calculate_log2(group):
    """
    Calculate the log2 of 'Smoothed_OD_Blank_Sub' for a given group of rows.
    
    Parameters:
        group (DataFrame): A subset of the DataFrame corresponding to a specific 'Sample' group.
    
    Returns:
        DataFrame: The original group with an additional 'Log2' column.
    """
    group['Log2'] = np.log2(group['Smoothed_OD_Blank_Sub'])
    return group
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_widths

from scipy.signal import find_peaks, peak_widths
import numpy as np
import pandas as pd

from scipy.signal import find_peaks, peak_widths
import numpy as np
import pandas as pd
def extract_metrics(group):
    t = group['Time'].values
    y = group['Smoothed_OD_Blank_Sub'].values  # already smoothed

    # --- Basic growth metrics ---
    auc = np.trapz(y, t)

    max_od_idx = np.argmax(y)
    max_od = y[max_od_idx]
    max_od_time = t[max_od_idx]
    last_od = y[-1]

    dy_dt = np.gradient(y, t)
    max_slope = np.max(dy_dt)

    decline_region = dy_dt[max_od_idx:]
    decline_rate = np.mean(decline_region) if len(decline_region) > 0 else 0

    # --- Peaks with all properties explicitly computed ---
    peaks_idx, peak_props = find_peaks(
        y,
        distance=5,
        height=(None, None),        # ✅ ensures 'peak_heights' is returned
        prominence=(None, None),    # ensures prominences and bases are computed
        width=(None, None),         # ensures width metrics are computed
        threshold=None,
        plateau_size=None
    )
    num_peaks = len(peaks_idx)

    # Compute widths (needed for width-based metrics)
    widths_res = peak_widths(y, peaks_idx, rel_height=0.5)

    # --- Helper function to compute max and mean safely ---
    def max_mean(arr):
        if arr is None or len(arr) == 0:
            return np.nan, np.nan
        return np.nanmax(arr), np.nanmean(arr)

    var_dODdt = np.var(dy_dt)

    # --- AUCs over specific time ranges ---
    x, y_time = 0, 10
    z, w = 10, 14

    group1 = group[(group['Time'] >= x) & (group['Time'] <= y_time)]
    group2 = group[(group['Time'] >= z) & (group['Time'] <= w)]

    auc1 = np.trapz(group1['Smoothed_OD_Blank_Sub'], group1['Time'])
    auc2 = np.trapz(group2['Smoothed_OD_Blank_Sub'], group2['Time'])

    # --- Peak property summaries (max + mean) ---
    peak_height_max, peak_height_mean = max_mean(peak_props.get('peak_heights', []))
    peak_prom_max, peak_prom_mean = max_mean(peak_props.get('prominences', []))
    peak_left_base_max, peak_left_base_mean = max_mean(peak_props.get('left_bases', []))
    peak_right_base_max, peak_right_base_mean = max_mean(peak_props.get('right_bases', []))
    peak_width_max, peak_width_mean = max_mean(widths_res[0])
    peak_width_height_max, peak_width_height_mean = max_mean(widths_res[1])
    peak_left_ip_max, peak_left_ip_mean = max_mean(widths_res[2])
    peak_right_ip_max, peak_right_ip_mean = max_mean(widths_res[3])

    # --- Collect all results ---
    result = {
        'AUC': auc,
        'AUC1': auc1,
        'AUC2': auc2,
        'Max_OD': max_od,
        'Max_OD_Time (h)': max_od_time,  # convert minutes to hours
        'Final_OD': last_od,
        'Max_Slope': max_slope,
        'Decline_Rate': decline_rate,
        'Num_Peaks': num_peaks,
        'Var_dODdt': var_dODdt,
        'Peak_Height_Max': peak_height_max,
        'Peak_Height_Mean': peak_height_mean,
        'Peak_Prominence_Max': peak_prom_max,
        'Peak_Prominence_Mean': peak_prom_mean,
        'Peak_Left_Base_Max': peak_left_base_max,
        'Peak_Left_Base_Mean': peak_left_base_mean,
        'Peak_Right_Base_Max': peak_right_base_max,
        'Peak_Right_Base_Mean': peak_right_base_mean,
        'Peak_Width_Max': peak_width_max,
        'Peak_Width_Mean': peak_width_mean,
        'Peak_Width_Height_Max': peak_width_height_max,
        'Peak_Width_Height_Mean': peak_width_height_mean,
        'Peak_Left_IP_Max': peak_left_ip_max,
        'Peak_Left_IP_Mean': peak_left_ip_mean,
        'Peak_Right_IP_Max': peak_right_ip_max,
        'Peak_Right_IP_Mean': peak_right_ip_mean
    }

    return pd.Series(result)


def calculate_sliding_regressions(df, window_regression):
    """
    Calculate linear regressions of Log2 values according to time for each 'Sample' over sliding windows of a specified size.

    Args:
        df (pandas.DataFrame): The input DataFrame containing 'Sample', 'Time', and 'Log2' columns.
        window_regression (int): The size of the sliding window for the regression.

    Returns:
        pandas.DataFrame: A DataFrame containing the regression results for each sliding window.
    """
    regression_results = []

    for sample, sample_df in df.groupby('Sample'):
        for i in range(len(sample_df) - window_regression + 1):
            window_df = sample_df.iloc[i:i+window_regression]
            x = window_df['Time'].values
            y = window_df['Log2'].values

            # Calculate linear regression using np.polyfit
            slope, intercept = np.polyfit(x, y, 1)

            # Calculate residuals and average residuals
            y_fit = slope * x + intercept
            residuals = y - y_fit
            avg_residuals = np.mean(np.abs(residuals))

            # Calculate R-squared
            ss_res = np.sum((y - y_fit) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            r_squared = 1 - (ss_res / ss_tot)

            regression_results.append({
                'Sample': sample,
                'Start_Time': window_df['Time'].iloc[0],
                'End_Time': window_df['Time'].iloc[-1],
                'Start_Log2': window_df['Log2'].iloc[0],
                'End_Log2': window_df['Log2'].iloc[-1],
                'Slope': slope,
                'Intercept': intercept,
                'Avg_Residuals': avg_residuals,
                'R_squared': r_squared
            })

    return pd.DataFrame(regression_results)

def plot_sample_regressions(df_back, df_reg, sample_to_plot, plot_name):
    # Filter data for the specific sample
    sample_data = df_back[df_back['Sample'] == sample_to_plot]
    sample_regressions = df_reg[df_reg['Sample'] == sample_to_plot]

    # Create the plot
    plt.figure(figsize=(10, 6))

    # Plot the Log2 values
    plt.scatter(sample_data['Time'], sample_data['Log2'], facecolors='none', edgecolors='lightgrey', s=30, label='Log2 Values')

    # Create a color map
    cmap = mcolors.LinearSegmentedColormap.from_list("", ["grey", "red"])

    # Get the range of slopes
    min_slope, max_slope = sample_regressions['Slope'].min(), sample_regressions['Slope'].max()

    # Plot each regression line with color based on slope
    for _, row in sample_regressions.iterrows():
        start_time, end_time = row['Start_Time'], row['End_Time']
        slope, intercept = row['Slope'], row['Intercept']
        
        # Normalize the slope to get a value between 0 and 1 (if multiple slopes)
        if min_slope == max_slope:
            # Use red color for the slope
            norm_slope = slope  # or you could set it to a specific value like 1 or 0
            color = 'red'
        else:
            norm_slope = (slope - min_slope) / (max_slope - min_slope)
            color = cmap(norm_slope)
           
        x = [start_time, end_time]
        y = [intercept + slope * start_time, intercept + slope * end_time]
        plt.plot(x, y, color=color, alpha=0.7, linewidth=2)

    plt.xlabel('Time')
    plt.ylabel('Log2 of OD600nm')
    plt.title(f"{plot_name} {sample_to_plot}")
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_two_regressions(title_detail, df_log2, reg1_df, reg2_df, sample_to_plot):
    sample_data = df_log2[df_log2['Sample'] == sample_to_plot]
    reg1 = reg1_df[reg1_df['Sample'] == sample_to_plot]
    reg2 = reg2_df[reg2_df['Sample'] == sample_to_plot]

    plt.figure(figsize=(10, 6))
    plt.scatter(sample_data['Time'], sample_data['Log2'], facecolors='none', edgecolors='lightgrey', s=30, label='Log2 Values')

    # First exponential phase regression (black)
    if not reg1.empty:
        x1 = [reg1['Start_Time'].values[0], reg1['End_Time'].values[0]]
        y1 = [reg1['Intercept'].values[0] + reg1['Slope'].values[0] * x1[0],
              reg1['Intercept'].values[0] + reg1['Slope'].values[0] * x1[1]]
        plt.plot(x1, y1, color='black', linewidth=2, label='First Exponential Phase')

    # Secondary exponential phase regression (red)
    if not reg2.empty:
        x2 = [reg2['Start_Time'].values[0], reg2['End_Time'].values[0]]
        y2 = [reg2['Intercept'].values[0] + reg2['Slope'].values[0] * x2[0],
              reg2['Intercept'].values[0] + reg2['Slope'].values[0] * x2[1]]
        plt.plot(x2, y2, color='red', linewidth=2, label='Secondary Exponential Phase')

    plt.xlabel('Time')
    plt.ylabel('Log2 of OD600nm')
    plt.title(f'Best regressions {title_detail} for {sample_to_plot}')
    plt.legend()
    plt.grid(True)
    plt.show()

def refine_exponential_phase(top_rows_df, df_log2, thresh):
    refined_df = top_rows_df.copy()

    for idx, row in refined_df.iterrows():
        sample = row['Sample']
        residues_thresh = row['Avg_Residuals'] * thresh 

        sample_data = df_log2[df_log2['Sample'] == sample]
        current_start, current_end = row['Start_Time'], row['End_Time']

        # Expand window towards lower time values
        current_start = expand_window(sample_data, current_start, current_end, residues_thresh, direction='backward')

        # Expand window towards higher time values
        current_end = expand_window(sample_data, current_start, current_end, residues_thresh, direction='forward')

        # Update the refined dataframe
        refined_df.at[idx, 'Start_Time'] = current_start
        refined_df.at[idx, 'End_Time'] = current_end
        refined_df.at[idx, 'Start_Log2'] = get_log2_value(sample_data, current_start)
        refined_df.at[idx, 'End_Log2'] = get_log2_value(sample_data, current_end)
        refined_df.at[idx, 'Avg_Residuals'], refined_df.at[idx, 'Slope'], refined_df.at[idx, 'Intercept'] = calculate_regression_data(sample_data, current_start, current_end)

    return refined_df

def expand_window(data, current_start, current_end, threshold, direction):
    while True:
        if direction == 'backward':
            new_time = get_previous_time(data, current_start)
            if new_time is None or exceeds_threshold(data, new_time, current_end, threshold):
                break
            current_start = new_time
        elif direction == 'forward':
            new_time = get_next_time(data, current_end)
            if new_time is None or exceeds_threshold(data, current_start, new_time, threshold):
                break
            current_end = new_time
    return current_start if direction == 'backward' else current_end

def exceeds_threshold(data, start_time, end_time, threshold):
    return calculate_regression_data(data, start_time, end_time)[0] > threshold


def calculate_regression_data(data, start_time, end_time):
    window = data[(data['Time'] >= start_time) & (data['Time'] <= end_time)]
    slope, intercept, _, _, _ = stats.linregress(window['Time'], window['Log2'])
    residuals = window['Log2'] - (slope * window['Time'] + intercept)
    avg_residuals = np.mean(np.abs(residuals))
    return avg_residuals, slope, intercept


def get_log2_value(data, time):
    return data.loc[data['Time'] == time, 'Log2'].values[0]

def get_previous_time(data, current_time):
    previous_times = data[data['Time'] < current_time]['Time']
    return previous_times.max() if not previous_times.empty else None

def get_next_time(data, current_time):
    next_times = data[data['Time'] > current_time]['Time']
    return next_times.min() if not next_times.empty else None




In [ ]:
"""
===========================================================
import data
===========================================================
"""

In [ ]:
import pandas as pd
import re

# -------------------------------
# Define variables
# -------------------------------
sheet_name = 'Table All Cycles'
skiprows = 12
nrows = 170
time_format = 'XX h XX min'

# -------------------------------
# Define replicate files
# -------------------------------
files = [
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate1_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate1_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate1_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate1_xylose_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate1_xylose_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate1_xylose_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate2_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate2_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate2_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate2_xylose_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate2_xylose_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate2_xylose_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate3_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate3_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate3_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate3_xylose_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate3_xylose_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate3_xylose_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate4_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate4_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/crispri/plate4_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate4_xylose_rep1.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate4_xylose_rep2.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/xylose/plate4_xylose_rep3.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/251009_missingreps.xlsx",
    "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/Plate2_Rep4_andMissingrep.xlsx",
     "C:/Users/arnou/Documents/thesis/Resultaten/voorpythoncode/23_10_25Plate7missingreps.xlsx"
]

# -------------------------------
# Load metadata Excel with multiple sheets
# -------------------------------
file_path_meta = 'C:/Users/arnou/Documents/thesis/Resultaten/MetaData/MetaDataNew13.xlsx'  # full path including .xlsx
metadata_excel = pd.ExcelFile(file_path_meta)

dfs = []

for i, file_path in enumerate(files):
    # -------------------------------
    # Load each replicate
    # -------------------------------
    df_rep = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=[0, 1],
        index_col=None,
        skiprows=skiprows,
        nrows=nrows
    )

    # Flatten headers
    df_rep.columns = [col[0] for col in df_rep.columns]
    df_rep.columns.values[1] = "Time"
    df_rep = df_rep[df_rep.columns[1:]]

    # Melt to long format
    df_melted = pd.melt(df_rep, id_vars='Time', var_name='Sample', value_name='OD')

    
    df_melted["source_file"] = file_path

    # -------------------------------
    # Merge with corresponding metadata sheet
    # -------------------------------
    # Select the correct sheet (sheet1 → first file, sheet2 → second file, etc.)
    sheet_name_meta = metadata_excel.sheet_names[i]  # i-th sheet corresponds to i-th file
    df_meta = pd.read_excel(file_path_meta, sheet_name=sheet_name_meta, header=0, index_col=None)

    df_merged = df_melted.merge(
        df_meta,
        left_on="Sample",
        right_on="MetaData_Well",
        how="left"
    ).rename(columns={
        "Sample": "Well",
        "MetaData_gene": "Sample"
    })

    dfs.append(df_merged)

# -------------------------------
# Combine all replicates
# -------------------------------
df_final = pd.concat(dfs, ignore_index=True)

# -------------------------------
# Reformat time
# -------------------------------
if time_format == 'XX h XX min':
    df_final['Time'] = df_final['Time'].apply(convert_time_to_minutes)
elif time_format == 'decimal hour':
    df_final['Time'] = df_final['Time'] * 60
elif time_format == 'minutes':
    df_final = df_final.rename(columns={'Time': 'Time'})
else:
    print('Unknown time format.')

print("Final merged and reformatted data:")
print(df_final)
df_melted = df_final
print(df_melted)

# -------------------------------
# Select which plate(s) to analyze
# -------------------------------
# Options:
#   'all' → all plates together
#   ['Plaat_1'] → just plate 1
#   ['Plaat_1', 'Plaat_2'] → a combination of plates
plates_to_analyze = ['all']  # change this as needed

if plates_to_analyze != ['all']:
    df_filtered = df_final[df_final['MetaData_Plaat'].isin(plates_to_analyze)].copy()
else:
    df_filtered = df_final.copy()

print(f"Data selected for plates: {plates_to_analyze}")
print(df_filtered)

df_melted = df_filtered
print(df_melted)

In [ ]:
import matplotlib.pyplot as plt

# --- Step 1: Get unique samples alphabetically ---
samples = df_melted['Sample'].unique()  
n_rows = (len(samples) + 2) // 3  # Calculate number of rows for subplots

# --- Step 2: Create figure and subplots ---
fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5 * n_rows))
axes = axes.flatten()
print('melted')
print(df_melted)
# --- Step 3: Plot each sample ---
for i, sample in enumerate(samples):
    sample_data = df_melted[df_melted['Sample'] == sample]
    print('sample data')
    print(sample_data)

    # Plot each replicate in a different color
    for (rep,xylose), rep_data in sample_data.groupby(['rep','Xylose']):
        print(rep_data)
        axes[i].plot(rep_data['Time'], rep_data['OD'], label=f'Rep {rep} {xylose}', alpha=0.8)

    axes[i].set_title(sample)
    axes[i].set_xlabel('Time (min)')
    axes[i].set_ylabel('OD')
    axes[i].legend()

# --- Step 4: Remove any unused subplots ---
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# Optional: print sample names alphabetically
print('List of sample names (alphabetical):')
print(samples)

In [ ]:
# Define variables for analysis

# List of specific sample–replicate combinations to exclude
# Format: ('Sample name', replicate_number)
# Example: [('Strain name:  CAG74399_1; Gene target: no_sgRNA', 2), ('Strain name:  BEC00920; Gene target: gltX', 1)]


samples_to_remove = [
    ('Strain name:  BEC14530; Gene target: rnjA', 1, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC14530; Gene target: rnjA', 2, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC14530; Gene target: rnjA', 3, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 1, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 2, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 3, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 1, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 2, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 3, 'no xylose', 'Plaat_1'),
    ('Strain name:  BEC16750; Gene target: asd', 1, 'no xylose', 'Plaat_4'),
    ('Strain name:  BEC16750; Gene target: asd', 2, 'no xylose', 'Plaat_4'),
    ('Strain name:  BEC16750; Gene target: asd', 3, 'no xylose', 'Plaat_4'),
    ('Strain name:  BEC14530; Gene target: rnjA', 1, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC14530; Gene target: rnjA', 2, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC14530; Gene target: rnjA', 3, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 1, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 2, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC25290; Gene target: era', 3, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 1, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 2, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC01500; Gene target: rpsI', 3, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC16750; Gene target: asd', 1, 'xylose', 'Plaat_4'),
    ('Strain name:  BEC16750; Gene target: asd', 2, 'xylose', 'Plaat_4'),
    ('Strain name:  BEC16750; Gene target: asd', 3, 'xylose', 'Plaat_4'),
    ('Strain name:  BEC17370; Gene target: nrdI', 1, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC17370; Gene target: nrdI', 2, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC17370; Gene target: nrdI', 3, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC00920; Gene target: gltX', 1, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC00920; Gene target: gltX', 2, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC00920; Gene target: gltX', 3, 'xylose', 'Plaat_1'),
    ('Strain name:  BEC35740; Gene target: tagD', 1, 'xylose', 'Plaat_3'),
    ('Strain name:  BEC35740; Gene target: tagD', 2, 'xylose', 'Plaat_3'),
    ('Strain name:  BEC35740; Gene target: tagD', 3, 'xylose', 'Plaat_3'),
    ('Strain name:  CAG74399_15; Gene target: rpsN', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC15890; Gene target: plsX', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC18100; Gene target: parC', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC22840; Gene target: engA', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC28850; Gene target: rplT', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC15180; Gene target: murE', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC13300; Gene target: mgtE', 2, 'no xylose', 'Plaat_2'),
('Strain name:  BEC13300; Gene target: mgtE', 3, 'no xylose', 'Plaat_2'),
('Strain name:  BEC06070; Gene target: ydiP', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC22780; Gene target: folE', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC21810; Gene target: dfrA', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC01040; Gene target: rplJ', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC01080; Gene target: rpoC', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC15220; Gene target: murG', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC16040; Gene target: rplS', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC16580; Gene target: polC', 1, 'no xylose', 'Plaat_3'),
('Strain name:  BEC17910; Gene target: yneF', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC01360; Gene target: secY', 2, 'xylose', 'Plaat_4'),
('Strain name:  BEC01430; Gene target: rpoA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  BEC15290.226; Gene target: ftsZ226', 2, 'xylose', 'Plaat_4'),
('Strain name:  BEC15230; Gene target: murB', 3, 'no xylose', 'Plaat_4'),
('Strain name:  BEC15940; Gene target: smc', 2, 'xylose', 'Plaat_4'),
('Strain name:  BEC15990; Gene target: rpsP', 3, 'xylose', 'Plaat_4'),
('Strain name:  BEC24310; Gene target: folD', 3, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_89; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 2, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_12; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_39; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_56; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_59; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_2'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 2, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_62; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_2'),
('Strain name:  CAG74399_75; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_75; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_75; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_77; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_83; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_84; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 2, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_86; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  BEC16550; Gene target: dxr', 1, 'no xylose', 'Plaat_1'),
('Strain name:  BEC16550; Gene target: dxr', 2, 'no xylose', 'Plaat_1'),
('Strain name:  BEC16550; Gene target: dxr', 3, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28850; Gene target: rplT', 1, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28850; Gene target: rplT', 2, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28850; Gene target: rplT', 3, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 1, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 2, 'no xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 3, 'no xylose', 'Plaat_1'),
('Strain name:  BEC01500; Gene target: rpsI', 1, 'no xylose', 'Plaat_1'),
('Strain name:  BEC01500; Gene target: rpsI', 2, 'no xylose', 'Plaat_1'),
('Strain name:  BEC01500; Gene target: rpsI', 3, 'no xylose', 'Plaat_1'),
('Strain name:  BEC14530; Gene target: rnjA', 1, 'no xylose', 'Plaat_1'),
('Strain name:  BEC14530; Gene target: rnjA', 2, 'no xylose', 'Plaat_1'),
('Strain name:  BEC14530; Gene target: rnjA', 3, 'no xylose', 'Plaat_1'),
('Strain name:  BEC31650; Gene target: mrpF', 1, 'no xylose', 'Plaat_2'),
('Strain name:  BEC31650; Gene target: mrpF', 2, 'no xylose', 'Plaat_2'),
('Strain name:  BEC31650; Gene target: mrpF', 3, 'no xylose', 'Plaat_2'),
('Strain name:  BEC32670; Gene target: sufB', 1, 'no xylose', 'Plaat_2'),
('Strain name:  BEC32670; Gene target: sufB', 2, 'no xylose', 'Plaat_2'),
('Strain name:  BEC32670; Gene target: sufB', 3, 'no xylose', 'Plaat_2'),
('Strain name:  BEC01320.2; Gene target: rplR-2', 1, 'no xylose', 'Plaat_3'),
('Strain name:  BEC01320.2; Gene target: rplR-2', 2, 'no xylose', 'Plaat_3'),
('Strain name:  BEC01320.2; Gene target: rplR-2', 3, 'no xylose', 'Plaat_3'),
('Strain name:  BEC01340; Gene target: rpmD', 1, 'no xylose', 'Plaat_3'),
('Strain name:  BEC01340; Gene target: rpmD', 2, 'no xylose', 'Plaat_3'),
('Strain name:  BEC01340; Gene target: rpmD', 3, 'no xylose', 'Plaat_3'),
('Strain name:  BEC17380; Gene target: nrdE', 1, 'no xylose', 'Plaat_3'),
('Strain name:  BEC17380; Gene target: nrdE', 2, 'no xylose', 'Plaat_3'),
('Strain name:  BEC17380; Gene target: nrdE', 3, 'no xylose', 'Plaat_3'),
('Strain name:  BEC25200; Gene target: sigA', 1, 'no xylose', 'Plaat_4'),
('Strain name:  BEC25200; Gene target: sigA', 2, 'no xylose', 'Plaat_4'),
('Strain name:  BEC25200; Gene target: sigA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  BEC01770; Gene target: glmM', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC01770; Gene target: glmM', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC01770; Gene target: glmM', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC15910; Gene target: fabG', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC15910; Gene target: fabG', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC15910; Gene target: fabG', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC25270; Gene target: glyQ', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC25270; Gene target: glyQ', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC25270; Gene target: glyQ', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC28870; Gene target: infC', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC_TRNA_23; Gene target: trnH', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC_TRNA_23; Gene target: trnH', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC_TRNA_23; Gene target: trnH', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC00460; Gene target: ipk', 1, 'xylose', 'Plaat_1'),
('Strain name:  BEC00460; Gene target: ipk', 2, 'xylose', 'Plaat_1'),
('Strain name:  BEC00460; Gene target: ipk', 3, 'xylose', 'Plaat_1'),
('Strain name:  BEC01500; Gene target: rpsI', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC01500; Gene target: rpsI', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC01500; Gene target: rpsI', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC00380; Gene target: metS', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC00380; Gene target: metS', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC00380; Gene target: metS', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC16530; Gene target: uppS', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC16530; Gene target: uppS', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC16530; Gene target: uppS', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC27560; Gene target: hisS', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC27560; Gene target: hisS', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC27560; Gene target: hisS', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC14190; Gene target: dapL', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC14190; Gene target: dapL', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC14190; Gene target: dapL', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC22420; Gene target: panC', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC22420; Gene target: panC', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC22420; Gene target: panC', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC16920; Gene target: pgsA', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC16920; Gene target: pgsA', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC16920; Gene target: pgsA', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC22490; Gene target: dapB', 1, 'xylose', 'Plaat_2'),
('Strain name:  BEC22490; Gene target: dapB', 2, 'xylose', 'Plaat_2'),
('Strain name:  BEC22490; Gene target: dapB', 3, 'xylose', 'Plaat_2'),
('Strain name:  BEC15930; Gene target: rnc', 1, 'xylose', 'Plaat_3'),
('Strain name:  BEC15930; Gene target: rnc', 2, 'xylose', 'Plaat_3'),
('Strain name:  BEC15930; Gene target: rnc', 3, 'xylose', 'Plaat_3'),
('Strain name:  BEC29660; Gene target: rpsD', 1, 'xylose', 'Plaat_4'),
('Strain name:  BEC29660; Gene target: rpsD', 2, 'xylose', 'Plaat_4'),
('Strain name:  BEC29660; Gene target: rpsD', 3, 'xylose', 'Plaat_4'),
('Strain name:  BEC32110; Gene target: yumC', 1, 'xylose', 'Plaat_4'),
('Strain name:  BEC32110; Gene target: yumC', 2, 'xylose', 'Plaat_4'),
('Strain name:  BEC32110; Gene target: yumC', 3, 'xylose', 'Plaat_4'),
('Strain name:  leeg; Gene target: nothing', 1, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 2, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 3, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 1, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 2, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 3, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 1, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 2, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 3, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 1, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 2, 'xylose', 'Plate_fout'),
('Strain name:  leeg; Gene target: nothing', 3, 'xylose', 'Plate_fout'),
('Strain name:  CAG74399_83; Gene target: no_sgRNA', 1, 'xylose', 'Plaat_3'),
('Strain name:  CAG74399_83; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_3'),
('Strain name:  CAG74399_83; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_3'),
('Strain name:  CAG74399_39; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_39; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_57; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_57; Gene target: no_sgRNA', 1, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_57; Gene target: no_sgRNA', 2, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_57; Gene target: no_sgRNA', 3, 'no xylose', 'Plaat_4'),
('Strain name:  CAG74399_89; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_4'),
('Strain name:  CAG74399_89; Gene target: no_sgRNA', 3, 'xylose', 'Plaat_4'),

]




    
samples_to_remove_all = ["Strain name:  SGL999; Gene target: nothing","Strain name:  leeg; Gene target: nothing","Strain name:  xylosevergeten; Gene target: fout"]


# If you want to remove *all* replicates of a sample, you can also list it as just the sample name
#samples_to_remove_all = [
    # 'Strain name:  BAD_SAMPLE; Gene target: something'


# Example of setting diauxic samples (optional)
# samples_diauxic = ['Strain name:  ABC123; Gene target: foo']

# Example of a sample to plot in detail later
sample_to_plot = 'Strain name:  BEC22600; Gene target: aroE'

# Parameters
window_size = 50          # Smoothing window for Savitzky–Golay filter
window_regression = 23   # Window size for best linear regression fit


In [ ]:
#export_path = 'C:/Users/arnou/Documents/thesis/Resultaten/exportfiles/analysisy.xlsx'


#df_melted.to_excel(export_path, index=False, float_format='%.12g')
#print(f"File successfully exported to {export_path}")

df_melted = df_melted[
    ~df_melted.set_index(['Sample', 'rep', 'Xylose', 'MetaData_Plaat']).index.isin(samples_to_remove)
]
df_melted = df_melted[
    ~df_melted.apply(lambda row: (row["Sample"]) in samples_to_remove_all, axis=1)
]

#('Strain name:  CAG74399_12; Gene target: no_sgRNA', 2, 'xylose', 'Plaat_2')

#export_path = 'C:/Users/arnou/Documents/thesis/Resultaten/exportfiles/analysisyy.xlsx'


#df_melted.to_excel(export_path, index=False, float_format='%.12g')
#print(f"File successfully exported to {export_path}")



In [ ]:
"""
===========================================================
Preprocessing
===========================================================
"""

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import numpy as np

# --- Smoothing using the Savitzky-Golay filter ---

# Group by both Sample and replicate
df_smoothed = df_melted.groupby(['Sample', 'rep','Xylose']).apply(
    apply_savgol_filter,
    window_size=window_size,
    polyorder=1
).reset_index(drop=True)

print(df_smoothed.head())

# --- Check-up plot for smoothing step ---

if sample_to_plot in df_smoothed['Sample'].unique():
    fig, ax = plt.subplots(figsize=(10, 6))

    for (rep,xylose), rep_data in df_smoothed[df_smoothed['Sample'] == sample_to_plot].groupby(['rep', 'Xylose']):
        ax.plot(rep_data['Time'], rep_data['OD'], linestyle='--', marker='o', alpha=0.6, label=f'Raw Rep {rep}{xylose}')
        ax.plot(rep_data['Time'], rep_data['Smoothed_OD'], marker='o', label=f'Smoothed Rep {rep}')

    ax.set_title(f'OD before and after smoothing for {sample_to_plot}')
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('OD 600nm')
    ax.legend()
    plt.show()
else:
    print('The specified sample is not present in the smoothed data.')


# --- Blank calculation (per sample–replicate) ---

filtered_df = df_smoothed[(df_smoothed['Time'] >= 1) & (df_smoothed['Time'] <= 15)]

minimum_smoothed_od = (
    filtered_df.groupby(['Sample', 'rep','Xylose'])['Smoothed_OD'].min().rename('Blank_value')
)

df_smoothed = df_smoothed.merge(minimum_smoothed_od, on=['Sample', 'rep','Xylose'], how='left')

print("Blank values for each sample-replicate:")
print(minimum_smoothed_od)
print('-' * 65)

# --- Subtract blank values ---

df_blank_subs = df_smoothed.copy()
df_blank_subs['Smoothed_OD_Blank_Sub'] = df_blank_subs['Smoothed_OD'] - df_blank_subs['Blank_value']

negative_values_count = (df_blank_subs['Smoothed_OD_Blank_Sub'] < 0).sum()
print(f"Number of negative, blank-subtracted OD values: {negative_values_count}")

#all smoothed plots
# --- Plot curves after smoothing and blank subtraction ---

import matplotlib.pyplot as plt

# Get unique samples
samples = df_blank_subs['Sample'].unique()
n_rows = (len(samples) + 2) // 3  # Calculate number of rows for subplots

# Create figure and subplots
fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5 * n_rows))
axes = axes.flatten()

# Plot each sample
for i, sample in enumerate(samples):
    sample_data = df_blank_subs[df_blank_subs['Sample'] == sample]

    # Plot each replicate
    for (rep,xylose), rep_data in sample_data.groupby(['rep', 'Xylose']):
        axes[i].plot(
            rep_data['Time'],
            rep_data['Smoothed_OD_Blank_Sub'],
            label=f'Rep {rep}{xylose}',
            alpha=0.8
        )

    axes[i].set_title(f'{sample}')
    axes[i].set_xlabel('Time (hours)')
    axes[i].set_ylabel('OD ')
    axes[i].legend()
    #axes[i].set_xlim([0, 15])
    #axes[i].set_ylim([0, y_max])

# Remove unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

print('List of sample names (smoothed + blank-subtracted):')
print(list(samples))


# --- Check-up plot for blank subtraction ---


if sample_to_plot in df_blank_subs['Sample'].unique():
    fig, ax = plt.subplots(figsize=(10, 6))
    for (rep,xylose), rep_data in df_blank_subs[df_blank_subs['Sample'] == sample_to_plot].groupby(['rep', 'Xylose']):
        ax.plot(rep_data['Time'], rep_data['Smoothed_OD_Blank_Sub'], marker='o', label=f'Rep {rep}')
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('Blank-subtracted OD')
    ax.set_title(f'Blank-subtracted OD for {sample_to_plot}')
    ax.legend()
    ax.grid(True)
    plt.show()
else:
    print('The specified sample is not present in this acquisition.')


In [ ]:
num_strains = df_smoothed['Sample'].nunique()
print(f"Number of different strains (unique Sample values): {num_strains}")


In [ ]:
num_strains = df_smoothed['Sample'].nunique()
print(f"Number of different strains (unique Sample values): {num_strains}")

# --- count no_sgRNA vs others ---
sample_list = df_smoothed['Sample'].unique()

num_no_sgRNA = sum("no_sgRNA".lower() in s.lower() for s in sample_list)
num_with_sgRNA = num_strains - num_no_sgRNA

print(f"Strains with no_sgRNA: {num_no_sgRNA}")
print(f"Strains with sgRNA (mutants): {num_with_sgRNA}")


In [ ]:
"""
===========================================================
Visualization and verification
===========================================================
"""

In [ ]:
# Function to plot mean growth curves with standard deviation as a shaded area
def plot_gene_targets_mean_std(df, gene_targets, time_limit=(0, 14), figsize=(10,6)):
    """
    For each gene target, plot the mean growth curve (line) and standard deviation (shaded area)
    for replicates with and without xylose on the same plot.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.lines import Line2D
    from matplotlib.patches import Patch
    
    if not isinstance(gene_targets, (list, tuple)):
        gene_targets = [gene_targets]
    targets_lower = gene_targets

    # Helper function to ensure Xylose_flag column exists
    if 'Xylose_flag' not in df.columns:
        def _is_xylose_val(v):
            try:
                s = str(v).strip().lower()
                return s in ("true", "yes", "1", "xylose", "y")
            except Exception:
                return False
        df = df.copy()
        df['Xylose_flag'] = df['Xylose'].apply(_is_xylose_val)

    # --- FONT SIZE DEFINITIONS (DOUBLED) ---
    TITLE_FONT = 24
    LABEL_FONT = 20
    TICK_FONT = 16
    LEGEND_FONT = 16

    for target in targets_lower:
        mask = df['Sample'].astype(str).str.contains(target)
        if mask.sum() == 0:
            print(f'No samples match gene target: {target}')
            continue
            
        sel = df[mask].copy()

        # Group data by Time and Xylose_flag to calculate mean and std
        grouped_data = sel.groupby(['Xylose_flag', 'Time'])['Smoothed_OD'].agg(['mean', 'std']).reset_index()

        # Separate data for 'With xylose' and 'Without xylose'
        data_xylose = grouped_data[grouped_data['Xylose_flag'] == True]
        data_no_xylose = grouped_data[grouped_data['Xylose_flag'] == False]
        
        fig, ax = plt.subplots(figsize=figsize)
        
        # --- Plot With Xylose (Blue) ---
        if not data_xylose.empty:
            time_xylose = data_xylose['Time']
            mean_xylose = data_xylose['mean']
            std_xylose = data_xylose['std'].fillna(0)
            
            ax.plot(time_xylose, mean_xylose, color='blue', linewidth=2.5, label='xylose')
            ax.fill_between(
                time_xylose, 
                mean_xylose - std_xylose, 
                mean_xylose + std_xylose, 
                color='blue', 
                alpha=0.2, 
                edgecolor='none'
            )

        # --- Plot Without Xylose (Green) ---
        if not data_no_xylose.empty:
            time_no_xylose = data_no_xylose['Time']
            mean_no_xylose = data_no_xylose['mean']
            std_no_xylose = data_no_xylose['std'].fillna(0)
            
            ax.plot(time_no_xylose, mean_no_xylose, color='green', linewidth=2.5, label='no xylose')
            ax.fill_between(
                time_no_xylose, 
                mean_no_xylose - std_no_xylose, 
                mean_no_xylose + std_no_xylose, 
                color='green', 
                alpha=0.2, 
                edgecolor='none'
            )

        # --- APPLY DOUBLED FONT SIZES ---
        ax.set_xlabel('Time (hours)', fontsize=LABEL_FONT)
        ax.set_ylabel('OD', fontsize=LABEL_FONT)
        ax.set_title(f'{target}', fontsize=TITLE_FONT, pad=15)
        ax.set_xlim(time_limit)
        
        # Axis numbers (Ticks)
        ax.tick_params(axis='both', which='major', labelsize=TICK_FONT)

        # Legend Styling - Box Restored
        legend_elements = [
            Line2D([0], [0], color='blue', lw=3, label='Mean xylose'),
            Patch(facecolor='blue', alpha=0.3, label='SD xylose'),
            Line2D([0], [0], color='green', lw=3, label='Mean no xylose'),
            Patch(facecolor='green', alpha=0.3, label='SD no xylose')
        ]
        
        # frameon=True (default) keeps the box
        ax.legend(handles=legend_elements, loc='upper left', fontsize=LEGEND_FONT, frameon=True)
        
        plt.tight_layout()
        
        # Save figure with target-based filename
        safe_name = target.replace(" ", "_")
        plt.savefig(f"{safe_name}_mean_std_xylose.svg", format="svg", bbox_inches='tight')
        plt.show()

# Run the function
plot_gene_targets_mean_std(df_smoothed, ['no_sgRNA', 'rghRA','pgm','sufS','dxr','dapL','coaBC','tagF','dfrA','mnaA','tagB','sunI','hbs', 'mreC', 'rpsB','ligA'])

In [ ]:
print(df_smoothed)
import pandas as pd
import re

# NOTE: Replace this section with your actual 'df_smoothed' DataFrame loading
# or ensure the following code is applied directly to your existing 'df_smoothed'.

# 1. Extract Strain Name and Gene Target using regular expressions
# Extract Strain name: captures everything between 'Strain name: ' and the following ';'
df_smoothed['Strain_Name'] = df_smoothed['Sample'].str.extract(r'Strain name:\s*(.*?);')
# Extract Gene target: captures everything after 'Gene target: '
df_smoothed['Gene_Target'] = df_smoothed['Sample'].str.extract(r'Gene target:\s*(.*)')

# Clean up any potential leading/trailing whitespace
df_smoothed['Strain_Name'] = df_smoothed['Strain_Name'].str.strip()
df_smoothed['Gene_Target'] = df_smoothed['Gene_Target'].str.strip()

# 2. Total number of unique samples (unique Strain Names)
unique_strains_count = df_smoothed['Strain_Name'].nunique()

# 3. List of all unique sample names (Strain Names)
unique_strain_names = df_smoothed['Strain_Name'].unique().tolist()

# 4. Count of unique strains that contain 'no_sgRNA' as gene target
# First, create a temporary DataFrame with only unique Strain/Target combinations
unique_strains_targets = df_smoothed[['Strain_Name', 'Gene_Target']].drop_duplicates()

# Then, filter this unique list for 'no_sgRNA' and count the strains
no_sgRNA_count = unique_strains_targets[unique_strains_targets['Gene_Target'] == 'no_sgRNA']['Strain_Name'].nunique()

print(f"Total Unique Strains Count: {unique_strains_count}")
print(f"List of Unique Strain Names: {unique_strain_names}")
print(f"Unique Strains with 'no_sgRNA' Gene Target Count: {no_sgRNA_count}")

In [ ]:
# Additional plotting helpers: specify gene targets and combined xylose/no-xylose plots
import matplotlib.pyplot as plt
import numpy as np
import re

# Robust detection of Xylose presence (handles booleans, ints and several strings)
def _is_xylose_val(v):
    try:
        s = str(v).strip().lower()
        return s in ("true", "yes", "1", "xylose", "y")
    except Exception:
        return False

# Add a boolean column that flags the presence of xylose if not already present
if 'Xylose_flag' not in df_blank_subs.columns:
    df_blank_subs['Xylose_flag'] = df_blank_subs['Xylose'].apply(_is_xylose_val)

# Regex patterns
_no_sg_patterns = re.compile(r'no[_\-\s]?s?g?r?n?a?', flags=re.IGNORECASE)
_sgl999_pattern = re.compile(r'sgl999', flags=re.IGNORECASE)

def is_no_sgRNA(sample):
    return bool(_no_sg_patterns.search(str(sample)))


def plot_all_with_and_without_xylose(df, time_limit=(0,14), figsize=(12,8)):
    """
    Produces two figures:
      - all samples WITH xylose
      - all samples WITHOUT xylose

    Behavior:
      - no_sgRNA samples are combined into ONE group → mean + SD shaded area (red)
      - all mutants plotted individually (gray or orange)
      - no_sgRNA plotted LAST (higher zorder) so it overlays the others
    """
    d = df.copy()
    if 'Xylose_flag' not in d.columns:
        d['Xylose_flag'] = d['Xylose'].apply(_is_xylose_val)

    groups = {
        'xylose': d[d['Xylose_flag'] == True],
        'no xylose': d[d['Xylose_flag'] == False]
    }

    def color_by_substring(sample):
        s = str(sample).lower()
        if 'sgl999' in s:
            return 'orange'
        return 'gray'

    for title, subdf in groups.items():
        if subdf.empty:
            print(f'No samples found for group: {title}')
            continue

        plt.figure(figsize=figsize)

        # --- 1) PLOT ALL OTHER SAMPLES FIRST (lower zorder = underneath) ---
        other_samples = [s for s in subdf['Sample'].unique() if not is_no_sgRNA(s)]

        for sample in other_samples:
            sample_data = subdf[subdf['Sample'] == sample]
            color = color_by_substring(sample)

            stats = sample_data.groupby('Time')['Smoothed_OD'].mean().reset_index()

            plt.plot(
                stats['Time'], stats['Smoothed_OD'],
                color=color, linewidth=1,
                zorder=1   # draw below no_sgRNA
            )

        # --- 2) PLOT no_sgRNA LAST (on top) ---
        nosg_df = subdf[subdf['Sample'].apply(is_no_sgRNA)]
        if not nosg_df.empty:
            nosg_stats = nosg_df.groupby('Time')['Smoothed_OD'].agg(['mean', 'std']).reset_index()
            time = nosg_stats['Time']
            mean = nosg_stats['mean']
            std = nosg_stats['std'].fillna(0)

            # SD shaded area (under the line but still above mutants)
            plt.fill_between(
                time, mean - std, mean + std,
                color='red',
                alpha=0.2,
                edgecolor='none',
                zorder=3   # above mutants but under the no_sgRNA line
            )

            # Solid mean line (drawn on top)
            plt.plot(
                time, mean,
                color='red', linewidth=2,
                zorder=4   # very top
            )

        # Legend
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='red', lw=2, label='no_sgRNA (mean ± SD)'),
            Line2D([0], [0], color='gray', lw=1, label='mutants'),
           
        ]

        plt.xlabel('Time (hours)')
        plt.ylabel('OD')
        plt.title(f'{title}')
        plt.xlim(time_limit)
        plt.ylim((0,1.35))
        plt.legend(handles=legend_elements, loc='upper right')

        plt.savefig(f"{title.replace(' ', '_')}.svg", format="svg")
        plt.show()


# Example usage:
plot_all_with_and_without_xylose(df_smoothed)


In [ ]:

"""
===========================================================
Regression and calculation of growth metrics
===========================================================
"""

In [ ]:


# --- Log2 calculation per replicate ---

df_log2 = (
    df_blank_subs.groupby(['Sample', 'rep','Xylose'])
    .apply(calculate_log2)
    .drop(columns=['OD', 'Smoothed_OD', 'Blank_value'])
    .reset_index(drop=True)
)

print(df_log2.head())

# --- Extract metrics per replicate ---

df_metrics = df_blank_subs.groupby(['Sample', 'rep', 'Xylose']).apply(extract_metrics).reset_index()
print(df_metrics.head())

# --- Check-up plot for Log2 calculation ---

if sample_to_plot in df_log2['Sample'].unique():
    fig, ax = plt.subplots(figsize=(10, 6))
    for (rep,xylose), rep_data in df_log2[df_log2['Sample'] == sample_to_plot].groupby(['rep', 'Xylose']):
        ax.plot(rep_data['Time'], rep_data['Log2'], marker='o', linestyle='-', label=f'Rep {rep}{xylose}')
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('Log2(OD)')
    ax.set_title(f'Log2(OD) for {sample_to_plot}')
    ax.legend()
    ax.grid(True)
    plt.show()
else:
    print('The specified sample is not present in the Log2-calculated data.')


# --- Drop invalid Log2 values ---

original_rows = len(df_log2)
df_log2 = df_log2.dropna(subset=['Log2'])
df_log2 = df_log2[df_log2['Log2'] != -np.inf]
new_rows = len(df_log2)

print(f"Number of rows dropped due to NaN or -inf in Log2 column: {original_rows - new_rows}")



In [ ]:
regression_df = (
    df_log2
    .groupby(['Sample','rep','Xylose'])
    .apply(lambda g: calculate_sliding_regressions(g, window_regression)
           .assign(
               Sample=g['Sample'].iloc[0],
               rep=g['rep'].iloc[0], Xylose=g['Xylose'].iloc[0],
           ))
    .reset_index(drop=True)
)

print(regression_df)

In [ ]:
# Define the filtering values to select the best candidate for visible exponential phase

crit1_linearity = 0.99 # Used in step B of the filtering to discard regressions with a good linear fit (exponential phase should have a bad linear fit). Typically = 0.99
crit2_ODincr =  0.002 # Used in step C of first exponential phase filtering to discard regressions with low OD increase. Typically = 0.002
crit3_fit =  0.97 # Used in step D of first exponential phase filtering to discard regressions with bad fit (R-squared >= crit3_fit, typically = 0.99)

In [ ]:
# Make a copy of blank-subtracted OD and rename column to 'Log2' for regression reuse
df_od_for_regression = df_blank_subs.copy().rename(columns={'Smoothed_OD_Blank_Sub': 'Log2'})

# Run calculate_sliding_regressions on the OD values per sample–replicate, keeping 'xylose'
regression_OD_df = (
    df_od_for_regression
    .groupby(['Sample','rep','Xylose'])
    .apply(lambda g: calculate_sliding_regressions(g, window_regression)
           .assign(
               Sample=g['Sample'].iloc[0],
               rep=g['rep'].iloc[0], Xylose=g['Xylose'].iloc[0]
           ))
    .reset_index(drop=True)
)
print(regression_OD_df)

# Merge on Sample, rep, Start_Time, End_Time
merged = regression_df.merge(
    regression_OD_df[['Sample', 'rep', 'Xylose','Start_Time', 'End_Time', 'R_squared']],
    on=['Sample', 'rep','Xylose', 'Start_Time', 'End_Time'],
    suffixes=('', '_OD')
)

# Filter out candidates where the OD linear fit is better than the exponential fit
filtered_regression_df = merged[merged['R_squared_OD'] < merged['R_squared']]

# Further filter by user-defined linearity threshold
filtered_regression_df = filtered_regression_df[filtered_regression_df['R_squared_OD'] < crit1_linearity]

# Drop the extra column if desired
filtered_regression_df = filtered_regression_df.drop(columns=['R_squared_OD'])

# --- Summary prints & plots per replicate ---
print('Number of candidate exponential phases before check for exponential fit:')
print(regression_df.groupby(['Sample', 'rep', 'Xylose']).size())

plot_sample_regressions(df_log2, regression_df, sample_to_plot, "Candidates before linear check for")

print('Number of candidate exponential phases after check for exponential fit:')
print(filtered_regression_df.groupby(['Sample', 'rep', 'Xylose']).size())

plot_sample_regressions(df_log2, filtered_regression_df, sample_to_plot, "Candidates after linear check for")





In [ ]:
# Filtering of the candidate regressions to extract the exponential phase
# Uncomment and change the sample name to plot a different sample
# sample_to_plot = 'Sample X6'

sorted_df = filtered_regression_df.copy()

# Step C: Filter out candidate regressions with low OD differences
sorted_df['Start_OD'] = np.power(2, sorted_df['Start_Log2'])
sorted_df['End_OD'] = np.power(2, sorted_df['End_Log2'])
sorted_df['difference_OD'] = sorted_df['End_OD'] - sorted_df['Start_OD']

# Filter per replicate
sorted_df = sorted_df[sorted_df['difference_OD'] >= crit2_ODincr].drop('difference_OD', axis=1)

# Plot after first filtering step
plot_sample_regressions(df_log2, sorted_df, sample_to_plot, 'After removing low OD differences for')

# Step D: Keep candidate regressions with high fit scores
sorted_df = sorted_df[sorted_df['R_squared'] >= crit3_fit]

# Plot after second filtering step
plot_sample_regressions(df_log2, sorted_df, sample_to_plot, 'After keeping R-squared >= ' + str(crit3_fit) + ' for')

# Save candidates for downstream secondary exponential phase detection
sorted_df_secondary_expo = sorted_df.copy()

# Step E: Extract candidate regression with the highest slope per replicate
sorted_df = sorted_df.sort_values(by='Slope', ascending=False).reset_index(drop=True)
sorted_df['Rank_S'] = sorted_df.groupby(['Sample','rep', 'Xylose']).cumcount() + 1

# Select the top regression for each sample-replicate
top_rows_df = sorted_df[sorted_df['Rank_S'] == 1].drop('Rank_S', axis=1)

# Plot the final selection
plot_sample_regressions(df_log2, top_rows_df, sample_to_plot, 'Final selection for')

# Print the final result
print(top_rows_df)

# Check the values extracted for one of the samples
if sample_to_plot in df_smoothed['Sample'].unique():
    print(top_rows_df[(top_rows_df['Sample'] == sample_to_plot)])
else:
    print('The indicated sample to plot is not present in this acquisition.')






In [ ]:
# Refinement of the exponential phases
authorized_diff = 2  # Threshold for refinement (e.g., 1.1–3)
print(top_rows_df)
# Apply refinement per replicate
final_first_expo_df = (
    top_rows_df
    .groupby(['Sample', 'rep','Xylose'])
    .apply(lambda g: refine_exponential_phase(g, df_log2, authorized_diff))
    .reset_index(drop=True)
)
print('dit is final expo df')
print(final_first_expo_df)

# Plot the best regression for a specific sample
# Uncomment and change the sample name to plot a different sample
# sample_to_plot = 'Sample X20'

# Plot the regressions for the specified sample after refinement
plot_sample_regressions(df_log2, final_first_expo_df, sample_to_plot, 'After refinement - Exponential phase of ')

# Display the refined data for the selected sample if it exists in the final dataset
if sample_to_plot in final_first_expo_df['Sample'].unique():
    print(final_first_expo_df[final_first_expo_df['Sample'] == sample_to_plot])
else:
    print(f"The sample '{sample_to_plot}' is not present in the final dataset.")


In [ ]:
import matplotlib.pyplot as plt

# Unique strains
samples = df_blank_subs['Sample'].unique()
nrows = len(samples)
ncols = 2  # no xylose vs xylose

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 6 * nrows), squeeze=False)

for i, sample in enumerate(samples):
    # Subset data for this strain
    sample_data = df_blank_subs[df_blank_subs['Sample'] == sample]
    sample_expo = final_first_expo_df[final_first_expo_df['Sample'] == sample]

    # --- Plot no xylose ---
    sample_no_xylose = sample_data[sample_data['Xylose'] == 'no xylose']
    for rep, rep_data in sample_no_xylose.groupby('rep'):
        axs[i, 0].plot(rep_data['Time'], rep_data['Smoothed_OD_Blank_Sub'], 
                       label=f'Rep {rep}', alpha=0.7)
    
    # Highlight exponential phases
    for _, row in sample_expo[sample_expo['Xylose'] == 'no xylose'].iterrows():
        axs[i, 0].axvspan(row['Start_Time'], row['End_Time'], alpha=0.3, color='green')

    axs[i, 0].set_title(f'{sample} - No xylose')
    axs[i, 0].set_ylabel('Smoothed OD (Blank Subtracted)')
    axs[i, 0].legend(loc='upper left')

    # --- Plot with xylose ---
    sample_xylose = sample_data[sample_data['Xylose'] != 'no xylose']
    for rep, rep_data in sample_xylose.groupby('rep'):
        axs[i, 1].plot(rep_data['Time'], rep_data['Smoothed_OD_Blank_Sub'], 
                       label=f'Rep {rep}', alpha=0.7)
    
    for _, row in sample_expo[sample_expo['Xylose'] != 'no xylose'].iterrows():
        axs[i, 1].axvspan(row['Start_Time'], row['End_Time'], alpha=0.3, color='green')

    axs[i, 1].set_title(f'{sample} - With xylose')
    axs[i, 1].set_ylabel('Smoothed OD (Blank Subtracted)')
    axs[i, 1].legend(loc='upper left')

# Set common x-labels
for ax in axs[-1, :]:
    ax.set_xlabel('Time (minutes)')

plt.subplots_adjust(hspace=0.6, wspace=0.3)
plt.show()



In [ ]:
# Add secondary exponential phase data to the final_first_expo_df dataframe
# If no secondary phase is available yet, just use first phase
growth_data_df = final_first_expo_df.copy()
growth_data_df = growth_data_df.drop(columns=['Start_Log2', 'End_Log2'], errors='ignore')
print(growth_data_df.columns)

In [ ]:
"""
===========================================================
Visualization of growth characteristics (quantification)
===========================================================
"""

In [ ]:
# --- Calculate and add growth rates & generation times of the first exponential phase ---
print(growth_data_df.head())
print(df_metrics)

# Growth rate per hour and generation time (min)
growth_data_df['Growth_rate (.h-1)'] = growth_data_df['Slope'] * 60  
growth_data_df['Generation_time (min)'] = 1 / growth_data_df['Slope']

# --- Optional: handle 2nd exponential phase if present ---
# if 'Slope_2nd' in growth_data_df.columns:
#     growth_data_df['Growth_rate (.h-1)_2nd'] = growth_data_df['Slope_2nd'] * 60
#     growth_data_df['Generation_time (min)_2nd'] = 1 / growth_data_df['Slope_2nd']

# --- Merge growth_data_df with df_metrics ---
# Ensure both have 'Xylose' if available

# Merge growth_data_df with df_metrics on Sample, rep, and Xylose


merge_keys = ['Sample', 'rep', 'Xylose']

# Merge all metrics from df_metrics, now including peak summaries
growth_data_df = growth_data_df.merge(
    df_metrics[[
        'Sample', 'rep', 'Xylose',
        # AUC metrics
        'AUC', 'AUC1', 'AUC2',
        # OD metrics
        'Max_OD', 'Max_OD_Time (h)', 'Final_OD',
        'Max_Slope', 'Decline_Rate', 'Var_dODdt','Num_Peaks',
        # Peak summary metrics
        'Peak_Height_Max', 'Peak_Height_Mean',
        'Peak_Prominence_Max', 'Peak_Prominence_Mean',
        'Peak_Left_Base_Max', 'Peak_Left_Base_Mean',
        'Peak_Right_Base_Max', 'Peak_Right_Base_Mean',
        'Peak_Width_Max', 'Peak_Width_Mean',
        'Peak_Width_Height_Max', 'Peak_Width_Height_Mean',
        'Peak_Left_IP_Max', 'Peak_Left_IP_Mean',
        'Peak_Right_IP_Max', 'Peak_Right_IP_Mean'
    ]],
    on=merge_keys,
    how='left'  # keeps all rows from growth_data_df even if df_metrics is missing
)


# --- Reorder columns for clarity ---
columns_order = [
    'Sample', 'rep', 'Xylose',
    'Growth_rate (.h-1)', 'Generation_time (min)',
    'Start_OD', 'Start_Time', 'End_OD', 'End_Time',
    'Slope', 'Intercept', 'R_squared', 'Avg_Residuals',
    'AUC', 'AUC1', 'AUC2',
    'Max_OD', 'Max_OD_Time (h)', 'Final_OD',
    'Max_Slope', 'Decline_Rate', 'Num_Peaks', 'Var_dODdt', 
    # Peak summaries
    'Peak_Height_Max', 'Peak_Height_Mean',
    'Peak_Prominence_Max', 'Peak_Prominence_Mean',
    'Peak_Left_Base_Max', 'Peak_Left_Base_Mean',
    'Peak_Right_Base_Max', 'Peak_Right_Base_Mean',
    'Peak_Width_Max', 'Peak_Width_Mean',
    'Peak_Width_Height_Max', 'Peak_Width_Height_Mean',
    'Peak_Left_IP_Max', 'Peak_Left_IP_Mean',
    'Peak_Right_IP_Max', 'Peak_Right_IP_Mean'
]

# Reorder dataframe
growth_data_df = growth_data_df[columns_order]

# Reorder DataFrame columns
growth_data_df = growth_data_df[columns_order]

#

# --- Display final per-replicate, per-condition growth data ---
print("✅ Final growth data (per replicate and xylose condition):")
print(growth_data_df)


In [ ]:
import pandas as pd

# Ensure correct dtype for grouping
growth_data_df["Xylose"] = growth_data_df["Xylose"].str.strip().str.lower()

# Select numeric columns automatically
numeric_cols = growth_data_df.select_dtypes(include="number").columns

# Group by Sample and Xylose to compute mean and std
grouped = growth_data_df.groupby(["Sample", "Xylose"], as_index=False)

mean_df = grouped[numeric_cols].mean()
std_df = grouped[numeric_cols].std()

# Add identifiers for rep
mean_df["rep"] = "mean"
std_df["rep"] = "sd"

# Merge back the non-numeric columns
# Keep Sample and Xylose as in the group, fill other columns with NaN
mean_df = mean_df.reindex(columns=growth_data_df.columns)
std_df = std_df.reindex(columns=growth_data_df.columns)

# Combine all: original + mean + sd
growth_data_with_avgs = pd.concat([growth_data_df, mean_df, std_df], ignore_index=True)

# Sort for neatness
growth_data_with_avgs = growth_data_with_avgs.sort_values(["Sample", "Xylose", "rep"]).reset_index(drop=True)

print(growth_data_with_avgs)


In [ ]:
# --- Extract Gene target from the Sample column ---
def extract_gene_target(sample_str):
    match = re.search(r"Gene target:\s*([\w\-]+)", str(sample_str))
    return match.group(1) if match else sample_str

growth_data_with_avgs["Gene_target"] = growth_data_with_avgs["Sample"].apply(extract_gene_target)
print(growth_data_df)

In [ ]:
# Load the Excel file into a DataFrame
file_path_meta = 'C:/Users/arnou/Documents/thesis/Resultaten/MetaData/growth_category2.xlsx'
growth_category = pd.read_excel(file_path_meta)

# Merge the two DataFrames on the 'gene target X' column
merged_df3 = pd.merge(
    growth_data_with_avgs,
    growth_category,
    on='Gene_target',
    how='left'  # keep all rows from growth_data_with_avg
)
print(merged_df3)
# Fill missing growth profile entries with 'Not present'
merged_df3['Growth_category'] = merged_df3['Growth_category'].fillna('Not present')
growth_data_with_avgs = merged_df3
# Optional: inspect the result
print(growth_data_with_avgs.head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# --- Ensure xylose column is consistent ---
growth_data_with_avgs["Xylose"] = growth_data_with_avgs["Xylose"].str.strip().str.lower()

# --- Select only the mean and sd rows ---
mean_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "mean"].copy()
sd_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "sd"].copy()



with pd.option_context('display.max_columns', None):
    print(mean_df)

# --- Compute ratio of AUC2/AUC1 (mean and sd) ---
mean_df["AUC1/AUC2"] = mean_df["AUC1"] / mean_df["AUC2"]

sd_df["AUC1/AUC2"] = mean_df["AUC1/AUC2"] * np.sqrt(
    (sd_df["AUC1"] / mean_df["AUC1"])**2 +
    (sd_df["AUC2"] / mean_df["AUC2"])**2
)


mean_df["log(AUC1/AUC2)"] = np.log(mean_df["AUC1/AUC2"])

sd_df["log(AUC1/AUC2)"] = np.sqrt(
    (sd_df["AUC1"] / mean_df["AUC1"])**2 +
    (sd_df["AUC2"] / mean_df["AUC2"])**2
)


# --- Metrics to plot ---
metrics_to_plot = [
    "AUC", "AUC1", "AUC2", "AUC1/AUC2", "log(AUC1/AUC2)",
    "Max_OD", "Max_OD_Time (h)", "Final_OD",
    "Max_Slope", "Decline_Rate", "Num_Peaks", "Var_dODdt",  'Max_Slope', 'Decline_Rate', 'Num_Peaks',
    "Peak_Height_Max", "Peak_Height_Mean",
    "Peak_Prominence_Max", "Peak_Prominence_Mean",
    "Peak_Left_Base_Max", "Peak_Left_Base_Mean",
    "Peak_Right_Base_Max", "Peak_Right_Base_Mean",
    "Peak_Width_Max", "Peak_Width_Mean",
    "Peak_Width_Height_Max", "Peak_Width_Height_Mean",
    "Peak_Left_IP_Max", "Peak_Left_IP_Mean",
    "Peak_Right_IP_Max", "Peak_Right_IP_Mean"
]
print(mean_df)
def split_dataframe_evenly(df, n_parts=2):
    """Split dataframe into nearly equal chunks by number of rows."""
    df_sorted = df.sort_values("Gene_target").reset_index(drop=True)
    n = len(df_sorted)
    size = math.ceil(n / n_parts)
    return [df_sorted.iloc[i:i + size] for i in range(0, n, size)]

def plot_metric_in_chunks(df_subset, condition_label, metric_name, n_chunks=2):
    """Plot metric with equal-sized x-axis chunks and color by growth profile."""
    chunks = split_dataframe_evenly(df_subset, n_chunks)
    n_chunks = len(chunks)
    fig, axes = plt.subplots(n_chunks, 1, figsize=(14, 6 * n_chunks), sharey=True)

    if n_chunks == 1:
        axes = [axes]

    # Assign a distinct color for each growth profile
    profiles = df_subset["Growth_category_mean"].unique()
    colors = plt.cm.tab10.colors  # up to 10 colors
    profile_color_map = {profile: colors[i % len(colors)] for i, profile in enumerate(profiles)}

    for i, (ax, chunk) in enumerate(zip(axes, chunks)):
        for profile in profiles:
            profile_chunk = chunk[chunk["Growth_category_mean"] == profile]
            if profile_chunk.empty:
                continue
            ax.errorbar(
                x=profile_chunk["Gene_target"],
                y=profile_chunk[f"{metric_name}_mean"],
                yerr=profile_chunk[f"{metric_name}_sd"],
                fmt='o',
                ecolor='gray',
                capsize=5,
                markersize=6,
                linestyle='None',
                color=profile_color_map[profile],
                label=profile
            )
        ax.set_title(f"{metric_name} ({condition_label}) – Part {i+1}", fontsize=11)
        ax.set_xlabel("Gene_target", fontsize=9)
        ax.set_ylabel(metric_name, fontsize=9)
        ax.tick_params(axis='x', rotation=90, labelsize=8)
        ax.grid(True, linestyle="--", alpha=0.4)
        ax.legend(title="Growth profile", fontsize=9)

    plt.tight_layout()
    plt.show()


# --- Main plotting loop ---
for metric in metrics_to_plot:
    merged_stats = pd.merge(
        mean_df[["Sample", "Gene_target", "Xylose", "Growth_category", metric]],
        sd_df[["Sample", "Gene_target", "Xylose", "Growth_category", metric]],
        on=["Sample", "Gene_target", "Xylose"],
        suffixes=("_mean", "_sd")
    )
    print(merged_stats)
    merged_xylose = merged_stats[merged_stats["Xylose"] == "xylose"]
    merged_no_xylose = merged_stats[merged_stats["Xylose"] == "no xylose"]

    plot_metric_in_chunks(merged_xylose, "with xylose", metric)
    plot_metric_in_chunks(merged_no_xylose, "without xylose", metric)



In [ ]:
# --- Ensure xylose column is consistent ---
growth_data_with_avgs["Xylose"] = growth_data_with_avgs["Xylose"].str.strip().str.lower()

# --- Select only the mean and sd rows ---
mean_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "mean"]
sd_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "sd"]

# --- Merge means and SDs ---
merged_stats = pd.merge(
    mean_df[["Sample", "Xylose", "AUC"]],
    sd_df[["Sample", "Xylose", "AUC"]],
    on=["Sample", "Xylose"],
    suffixes=("_mean", "_sd")
)

# --- Separate xylose and no xylose ---
df_xylose = merged_stats[merged_stats["Xylose"] == "xylose"].set_index("Sample")
df_noxylose = merged_stats[merged_stats["Xylose"] == "no xylose"].set_index("Sample")

# --- Keep only samples that exist in both conditions ---
common_samples = df_xylose.index.intersection(df_noxylose.index)
df_xylose = df_xylose.loc[common_samples]
df_noxylose = df_noxylose.loc[common_samples]

# --- Compute ratio and propagated SD ---
ratio_mean = df_xylose["AUC_mean"] / df_noxylose["AUC_mean"]

# Error propagation for ratio (R = A/B):
# (σ_R / R)^2 = (σ_A / A)^2 + (σ_B / B)^2
ratio_sd = ratio_mean * np.sqrt(
    (df_xylose["AUC_sd"] / df_xylose["AUC_mean"])**2 +
    (df_noxylose["AUC_sd"] / df_noxylose["AUC_mean"])**2
)

# --- Combine into single dataframe ---
auc_ratio_df = pd.DataFrame({
    "Sample": common_samples,
    "AUC_ratio_mean": ratio_mean,
    "AUC_ratio_sd": ratio_sd
}).reset_index(drop=True)
# Add back Gene_target metadata
print(auc_ratio_df)

In [ ]:
import re

# --- extract gene target (leave values as-is, don't rename) ---
def extract_gene_target(sample_str):
    match = re.search(r"Gene target:\s*([\w\-]+)", str(sample_str))
    return match.group(1) if match else sample_str

auc_ratio_df["Gene_target"] = auc_ratio_df["Sample"].apply(extract_gene_target)

# --- prepare sorted DataFrame: controls (no_sgRNA) leftmost, others sorted desc ---
mask_no = auc_ratio_df["Gene_target"].str.lower() == "no_sgRNA".lower()

df_controls = auc_ratio_df[mask_no].copy().reset_index(drop=True)        # keep original order
df_others   = auc_ratio_df[~mask_no].sort_values("AUC_ratio_mean", ascending=False).reset_index(drop=True)

auc_sorted = pd.concat([df_controls, df_others], ignore_index=True)

# --- plotting: red for controls, black for others; x-axis labels = Gene_target (no renaming) ---
colors = ["red" if gt.lower() == "no_sgrna" else "black" for gt in auc_sorted["Gene_target"]]

plt.figure(figsize=(14,10))

# errorbars (gray)
plt.errorbar(
    x=range(len(auc_sorted)),                            # use numeric x for proper spacing when labels duplicate
    y=auc_sorted["AUC_ratio_mean"],
    yerr=auc_sorted["AUC_ratio_sd"],
    fmt='none',                                         # draw only errorbars here
    ecolor='gray',
    capsize=5,
    zorder=1
)

# points colored
plt.scatter(
    x=range(len(auc_sorted)),
    y=auc_sorted["AUC_ratio_mean"],
    c=colors,
    s=30,
    zorder=3
)

plt.axhline(1, color='black', linestyle='--', alpha=0.7, label='Ratio = 1 (equal AUC)')

# set x-ticks to gene target labels (duplicates allowed)
plt.xticks(ticks=range(len(auc_sorted)), labels=auc_sorted["Gene_target"], rotation=60, ha='right')

plt.title("AUC Ratio (xylose / no xylose) ± SD", fontsize=14)
plt.xlabel("Gene target", fontsize=12)
plt.ylabel("AUC Ratio", fontsize=12)
#plt.grid(True, linestyle="--", alpha=0.4)

# optional legend for colors
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='no_sgRNA'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=8, label='mutants'),
]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig("AUCratio_eerstnosgrna_danrest.svg", format="svg")
plt.show()



In [ ]:
mask_no = auc_ratio_df["Gene_target"].str.lower() == "no_sgRNA".lower()

# --- count strains ---
total_strains = len(auc_ratio_df)
num_no_sgRNA = mask_no.sum()
num_with_sgRNA = total_strains - num_no_sgRNA

print(f"Total strains: {total_strains}")
print(f"no_sgRNA strains: {num_no_sgRNA}")
print(f"With sgRNA strains: {num_with_sgRNA}")


In [ ]:


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

# --- Ensure xylose column is consistent ---
growth_data_with_avgs["Xylose"] = growth_data_with_avgs["Xylose"].str.strip().str.lower()

# --- Select only mean and sd rows ---
mean_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "mean"]
sd_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "sd"]

# --- Merge means and SDs for AUC ratio calculation ---
merged_stats = pd.merge(
    mean_df[["Sample", "Xylose", "AUC", "Growth_category"]],
    sd_df[["Sample", "Xylose", "AUC"]],
    on=["Sample", "Xylose"],
    suffixes=("_mean", "_sd")
)

# --- Separate xylose and no xylose ---
df_xylose = merged_stats[merged_stats["Xylose"] == "xylose"].set_index("Sample")
df_noxylose = merged_stats[merged_stats["Xylose"] == "no xylose"].set_index("Sample")

# --- Keep only samples present in both conditions ---
common_samples = df_xylose.index.intersection(df_noxylose.index)
df_xylose = df_xylose.loc[common_samples]
df_noxylose = df_noxylose.loc[common_samples]

# --- Compute AUC ratio and propagated SD ---
ratio_mean = df_xylose["AUC_mean"] / df_noxylose["AUC_mean"]
ratio_sd = ratio_mean * np.sqrt(
    (df_xylose["AUC_sd"] / df_xylose["AUC_mean"])**2 +
    (df_noxylose["AUC_sd"] / df_noxylose["AUC_mean"])**2
)

# --- Combine into AUC ratio dataframe ---
auc_ratio_df = pd.DataFrame({
    "Sample": common_samples,
    "AUC_ratio_mean": ratio_mean,
    "AUC_ratio_sd": ratio_sd,
    "Growth_category": df_xylose["Growth_category"].values
}).reset_index(drop=True)

# --- Extract Gene target from the Sample column ---
def extract_gene_target(sample_str):
    match = re.search(r"Gene target:\s*([\w\-]+)", str(sample_str))
    return match.group(1) if match else sample_str

auc_ratio_df["Gene_target"] = auc_ratio_df["Sample"].apply(extract_gene_target)

# --- Add AUC1/AUC2 log ratio ---
mean_df["AUC1/AUC2"] = mean_df["AUC1"] / mean_df["AUC2"]
mean_df["log(AUC1/AUC2)"] = np.log(mean_df["AUC1/AUC2"])

# --- Merge AUC ratio into mean_df ---
mean_df = mean_df.merge(
    auc_ratio_df[["Sample", "AUC_ratio_mean"]],
    on="Sample",
    how="left"
)

# --- Define metrics for PCA (including AUC ratio) ---
metrics_for_pca = [
    # --- AUC-related ---
   
   
    "log(AUC1/AUC2)",
    "AUC_ratio_mean",
    
    # --- OD (Optical Density) Metrics ---
    "Max_OD",
    "Max_OD_Time (h)",
    "Final_OD",
    
    # --- Peak Metrics ---
    "Peak_Prominence_Max"
]

# --- Filter mean_df for xylose condition and chosen metrics ---
pca_df = mean_df[mean_df["Xylose"] == "xylose"].copy()
pca_df = pca_df[["Sample", "Gene_target", "Growth_category"] + metrics_for_pca].set_index("Sample")

# --- Replace missing metric values with 0 instead of dropping ---
pca_df_clean = pca_df.copy()
pca_df_clean[metrics_for_pca] = pca_df_clean[metrics_for_pca].fillna(0)
# List strains where Peak_Prominence_Max is 0
zero_peak_strains = pca_df_clean[pca_df_clean["Peak_Prominence_Max"] == 0]

print("Number of strains with Peak_Prominence_Max = 0:", len(zero_peak_strains))
print(zero_peak_strains.index.tolist())




# --- Standardize metrics ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pca_df_clean[metrics_for_pca])

# --- Run PCA ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# --- Create PCA result dataframe ---
pca_result_df = pd.DataFrame(
    X_pca,
    columns=["PC1", "PC2"],
    index=pca_df_clean.index
)
pca_result_df["Gene_target"] = pca_df_clean["Gene_target"]

# --- Plot PCA with points in black except for no_sgRNA in red ---
import numpy as np
from scipy.spatial.distance import pdist, squareform

# --- Parameters for labeling ---
min_distance = 0.8   # distance threshold to consider "close"
max_cluster_size = 5  # clusters larger than this are not labeled
text_offset = 0.04    # offset for text from the points

positions = pca_result_df[["PC1", "PC2"]].to_numpy()
labels = pca_result_df["Gene_target"].to_numpy()

# Compute pairwise distances
dist_matrix = squareform(pdist(positions))

# Determine cluster sizes (number of neighbors within min_distance)
cluster_sizes = np.sum(dist_matrix < min_distance, axis=1) - 1  # subtract 1 to ignore self

# --- Plot PCA ---
plt.figure(figsize=(9, 7))

# Plot non-control samples (all genes except no_sgRNA) first
non_control = pca_result_df[pca_result_df["Gene_target"].str.lower() != "no_sgrna"]
plt.scatter(
    non_control["PC1"],
    non_control["PC2"],
    color='black',
    label='All genes',
    s=30,
    alpha=0.8,
    zorder=1  # background
)

# Plot control samples (no_sgRNA) last to appear in foreground
control = pca_result_df[pca_result_df["Gene_target"].str.lower() == "no_sgrna"]
if not control.empty:
    plt.scatter(
        control["PC1"],
        control["PC2"],
        color='red',
        label='no_sgRNA',
        s=30,
        alpha=0.8,
        zorder=2  # foreground
    )

# --- Add gene target labels with offset ---
for i, (x, y) in enumerate(positions):
    # Label if small cluster (<= max_cluster_size) or always for no_sgRNA, schrijf no_sgrna in keline letterso meht te laten werken
    # --- Add labels only for non-no_sgrna + small clusters ---

    if labels[i].lower() == "no_sgrna":
        continue

    if cluster_sizes[i] <= max_cluster_size or labels[i].lower() == "no_sgRNA":
        plt.text(x + text_offset, y + text_offset, labels[i], fontsize=8, alpha=0.8, zorder=3)

plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("PCA of Selected Metrics (xylose condition)")
#plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.savefig(f"pcaRed_BlackJuist1zonderauc.svg", format="svg")
plt.show()



print(f"Number of strains plotted in PCA: {len(pca_result_df)}")
print(f"Number of strains plotted in PCA zonder nosgrna: {len(pca_result_df[pca_result_df["Gene_target"].str.lower() != "no_sgrna"])}")


In [ ]:
#bigger font:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform

# --- GLOBAL FONT SETTINGS (Doubled) ---
plt.rcParams.update({
    'font.size': 18,          # Base font size
    'axes.titlesize': 24,      # Title font size
    'axes.labelsize': 20,      # X and Y label font size
    'xtick.labelsize': 16,     # X axis tick numbers
    'ytick.labelsize': 16,     # Y axis tick numbers
    'legend.fontsize': 16,     # Legend font size
    'figure.titlesize': 26     # Figure title
})

# [Existing data processing code remains the same...]
# (Keeping the logic until the plotting sections)

# --- Parameters for labeling ---
min_distance = 0.8
max_cluster_size = 5
text_offset = 0.04
gene_label_size = 16  # Doubled from 8

# ==========================
#     PCA SCATTER PLOT
#  Color = AUC_ratio_mean
# ==========================

plt.figure(figsize=(12, 9)) # Increased figure size slightly for larger fonts

color_values = pca_df_clean.loc[pca_result_df.index, "AUC_ratio_mean"]

scatter = plt.scatter(
    pca_result_df["PC1"],
    pca_result_df["PC2"],
    c=color_values,
    cmap="Reds",
    s=60, # Increased dot size slightly to match font
    alpha=0.9,
    edgecolors="black",
    linewidths=0.3
)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label("AUC ratio (xylose / no xylose)", size=20)
cbar.ax.tick_params(labelsize=16)

# --- Add gene target labels ---
for i, (x, y) in enumerate(positions):
    if labels[i].lower() == "no_sgrna":
        continue
    if cluster_sizes[i] <= max_cluster_size:
        # Changed fontsize to gene_label_size variable
        plt.text(x + text_offset, y + text_offset, labels[i], fontsize=gene_label_size, alpha=0.8)

plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("PCA of Selected Metrics (xylose condition)")
plt.tight_layout()
plt.savefig("pca_AUCratio_colored.svg", format="svg")
plt.show()


# ================================
#  GENERATE SEPARATE PCA PLOTS
# ================================

metrics_to_plot = [
    "log(AUC1/AUC2)",
    "AUC_ratio_mean",
    "Max_OD",
    "Max_OD_Time (h)",
    "Final_OD",
    "Peak_Prominence_Max"
]

for metric in metrics_to_plot:
    plt.figure(figsize=(12, 9))

    color_values = pca_df_clean.loc[pca_result_df.index, metric]

    scatter = plt.scatter(
        pca_result_df["PC1"],
        pca_result_df["PC2"],
        c=color_values,
        cmap="Reds",
        s=60,
        alpha=0.9,
        edgecolors="black",
        linewidths=0.3
    )

    cbar = plt.colorbar(scatter)
    cbar.set_label(metric, size=20)
    cbar.ax.tick_params(labelsize=16)

    for i, (x, y) in enumerate(positions):
        if cluster_sizes[i] <= max_cluster_size:
            plt.text(x + text_offset, y + text_offset, labels[i], fontsize=gene_label_size, alpha=0.8)

    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
    plt.title(f"PCA Colored by {metric}")

    plt.tight_layout()
    plt.savefig(f"PCA_{metric.replace('/', '_').replace(' ', '_')}.svg", format="svg")
    plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

# --- Ensure xylose column is consistent ---
growth_data_with_avgs["Xylose"] = growth_data_with_avgs["Xylose"].str.strip().str.lower()

# --- Select only mean and sd rows ---
mean_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "mean"]
sd_df = growth_data_with_avgs[growth_data_with_avgs["rep"] == "sd"]

# --- Merge means and SDs for AUC ratio calculation ---
merged_stats = pd.merge(
    mean_df[["Sample", "Xylose", "AUC", "Growth_category"]],
    sd_df[["Sample", "Xylose", "AUC"]],
    on=["Sample", "Xylose"],
    suffixes=("_mean", "_sd")
)

# --- Separate xylose and no xylose ---
df_xylose = merged_stats[merged_stats["Xylose"] == "xylose"].set_index("Sample")
df_noxylose = merged_stats[merged_stats["Xylose"] == "no xylose"].set_index("Sample")

# --- Keep only samples present in both conditions ---
common_samples = df_xylose.index.intersection(df_noxylose.index)
df_xylose = df_xylose.loc[common_samples]
df_noxylose = df_noxylose.loc[common_samples]

# --- Compute AUC ratio and propagated SD ---
ratio_mean = df_xylose["AUC_mean"] / df_noxylose["AUC_mean"]
ratio_sd = ratio_mean * np.sqrt(
    (df_xylose["AUC_sd"] / df_xylose["AUC_mean"])**2 +
    (df_noxylose["AUC_sd"] / df_noxylose["AUC_mean"])**2
)

# --- Combine into AUC ratio dataframe ---
auc_ratio_df = pd.DataFrame({
    "Sample": common_samples,
    "AUC_ratio_mean": ratio_mean,
    "AUC_ratio_sd": ratio_sd,
    "Growth_category": df_xylose["Growth_category"].values
}).reset_index(drop=True)

# --- Extract Gene target from the Sample column ---
def extract_gene_target(sample_str):
    match = re.search(r"Gene target:\s*([\w\-]+)", str(sample_str))
    return match.group(1) if match else sample_str

auc_ratio_df["Gene_target"] = auc_ratio_df["Sample"].apply(extract_gene_target)

# --- Add AUC1/AUC2 log ratio ---
mean_df["AUC1/AUC2"] = mean_df["AUC1"] / mean_df["AUC2"]
mean_df["log(AUC1/AUC2)"] = np.log(mean_df["AUC1/AUC2"])

# --- Merge AUC ratio into mean_df ---
mean_df = mean_df.merge(
    auc_ratio_df[["Sample", "AUC_ratio_mean"]],
    on="Sample",
    how="left"
)

# --- Define metrics for PCA (including AUC ratio) ---
metrics_for_pca = [
    # --- AUC-related ---
    
   
    "log(AUC1/AUC2)",
    "AUC_ratio_mean",
    
    # --- OD (Optical Density) Metrics ---
    "Max_OD",
    "Max_OD_Time (h)",
    "Final_OD",
    
    # --- Peak Metrics ---
    "Peak_Prominence_Max"
]

# --- Filter mean_df for xylose condition and chosen metrics ---
pca_df = mean_df[mean_df["Xylose"] == "xylose"].copy()
pca_df = pca_df[["Sample", "Gene_target", "Growth_category"] + metrics_for_pca].set_index("Sample")

# --- Drop samples with missing values in PCA metrics ---
# --- Replace missing metric values with 0 instead of dropping ---
pca_df_clean = pca_df.copy()
pca_df_clean[metrics_for_pca] = pca_df_clean[metrics_for_pca].fillna(0)
# List strains where Peak_Prominence_Max is 0
zero_peak_strains = pca_df_clean[pca_df_clean["Peak_Prominence_Max"] == 0]

print("Number of strains with Peak_Prominence_Max = 0:", len(zero_peak_strains))
print(zero_peak_strains.index.tolist())

# --- Standardize metrics ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pca_df_clean[metrics_for_pca])

# --- Run PCA ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# --- Create PCA result dataframe ---
pca_result_df = pd.DataFrame(
    X_pca,
    columns=["PC1", "PC2"],
    index=pca_df_clean.index
)
pca_result_df["Gene_target"] = pca_df_clean["Gene_target"]

# --- Plot PCA with points in black except for no_sgRNA in red ---
import numpy as np
from scipy.spatial.distance import pdist, squareform

# --- Parameters for labeling ---
min_distance = 0.8   # distance threshold to consider "close"
max_cluster_size = 5  # clusters larger than this are not labeled
text_offset = 0.04    # offset for text from the points

positions = pca_result_df[["PC1", "PC2"]].to_numpy()
labels = pca_result_df["Gene_target"].to_numpy()

# Compute pairwise distances
dist_matrix = squareform(pdist(positions))

# Determine cluster sizes (number of neighbors within min_distance)
cluster_sizes = np.sum(dist_matrix < min_distance, axis=1) - 1  # subtract 1 to ignore self

# --- Plot PCA ---
plt.figure(figsize=(9, 7))

# Plot non-control samples (all genes except no_sgRNA) first
non_control = pca_result_df[pca_result_df["Gene_target"].str.lower() != "no_sgrna"]
plt.scatter(
    non_control["PC1"],
    non_control["PC2"],
    color='black',
    label='All genes',
    s=30,
    alpha=0.8,
    zorder=1  # background
)

# Plot control samples (no_sgRNA) last to appear in foreground
control = pca_result_df[pca_result_df["Gene_target"].str.lower() == "no_sgrna"]
if not control.empty:
    plt.scatter(
        control["PC1"],
        control["PC2"],
        color='red',
        
        s=30,
        alpha=0.8,
        zorder=2  # foreground
    )
# --- Highlight specific gene targets ---
highlight_genes = ["dxr", "sufs", "pgm", "pheS","ylan27_1","ylan27_2","acps50_1","acps50_2", "ftsz226","ftsz","mreC"]  # <--- write small letters

highlight_df = pca_result_df[pca_result_df["Gene_target"].str.lower().isin(highlight_genes)]

# Plot highlighted points in cyan
plt.scatter(
    highlight_df["PC1"],
    highlight_df["PC2"],
    color='cyan',
    s=30,
    edgecolor='black',
    linewidth=0.8,
    zorder=5,
    label="Highlighted genes"
)

# Force labeling of highlighted genes
force_label_indices = highlight_df.index.tolist()
for i, (x, y) in enumerate(positions):
    gt = labels[i].lower()

    if (i in force_label_indices) or (cluster_sizes[i] <= max_cluster_size) or (gt == "no_sgrna"):
        plt.text(x + text_offset, y + text_offset, labels[i], fontsize=8, alpha=0.9, zorder=6)



plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("PCA of Selected Metrics (xylose condition)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.savefig(f"pcaRed_BlackJuist1.svg", format="svg")
plt.show()
